# Estudiante: Roy Chavarría Garita
## Semana #3

### Ejercicios
-Replicar lo visto en el video https://youtu.be/oYBHzSs7eRo

In [1]:
###Bibliotecas
#nn es nuronal networks
import torch.nn as nn
import numpy as np
import torch

In [2]:
#inputs = (temperatura, precipitacion, humedad)
inputs = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70],
                  [74,66,43],
                  [91,87,65],
                  [88,134,59],
                  [101,44,37],
                  [68,96,71],
                  [73,66,44],
                  [92,87,64],
                  [87,135,57],
                  [103,43,36],
                  [68,97,70]], dtype='float32')

#targets (manzanas, naranjas)

targets = np.array([[56,70],
                    [81,101],
                    [119,133],
                    [22,37],
                    [185,119],
                    [57,69],
                    [82,102],
                    [118,132],
                    [21,38],
                    [104,118],
                    [57,69],
                    [82,100],
                    [118,134],
                    [20,38],
                    [102,120]],dtype='float32')


inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print(inputs.size())
print(targets.size())

torch.Size([15, 3])
torch.Size([15, 2])


In [3]:
## Datase
#Ayuda a crear un conjunto de entrenamiento, en donde se encuentran inputs y tragets

In [4]:
from torch.utils.data import TensorDataset

In [5]:
train_dataset = TensorDataset(inputs,targets)

In [6]:
#EL trainset es un par ordenado en donde el par en la 0 son los inputs y en la posicion 1 los targest
print(train_dataset[0])

(tensor([73., 67., 43.]), tensor([56., 70.]))


## DataLoader

Clase que permite cargar datos que estan en el dataset y así poder prepararlos para entrenamiento

In [7]:
from torch.utils.data.dataloader import DataLoader
 #Definir el dataloader
#batch_size = 15
train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)

In [8]:
for input, target in train_dataloader:
  print(input)
  print(target)
  break

tensor([[ 87., 135.,  57.],
        [ 73.,  67.,  43.],
        [103.,  43.,  36.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.]])
tensor([[118., 134.],
        [ 56.,  70.],
        [ 20.,  38.],
        [118., 132.],
        [ 21.,  38.]])


Un dataloader 
EL batch size es para recorrer el dataloader de 5 en 5.
El shuffle es para que me agarre los datos de forma aleatoria.

In [9]:
#Esta función ya crea los pesos y sesgos de forma aleatoria
# 3 es la cantida de entradas de un input y 2 la de los targets
model = nn.Linear(3,2)

print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1272, -0.2572,  0.1031],
        [ 0.2556, -0.2437,  0.0124]], requires_grad=True)
Parameter containing:
tensor([-0.4048,  0.5063], requires_grad=True)


In [10]:
#Otra froma de ver los pesos y sesgos es 
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1272, -0.2572,  0.1031],
         [ 0.2556, -0.2437,  0.0124]], requires_grad=True),
 Parameter containing:
 tensor([-0.4048,  0.5063], requires_grad=True)]

In [11]:
#Genero las predicciones
predicciones = model(inputs)
print(predicciones)

tensor([[ -3.9181,   3.3707],
        [ -4.8641,   3.1147],
        [-17.8231,  -9.1925],
        [  5.3240,  16.5566],
        [ -9.1008,  -4.3830],
        [ -3.5338,   3.8700],
        [ -4.5038,   3.3708],
        [-17.5927,  -8.9245],
        [  4.9397,  16.0573],
        [ -9.1248,  -4.6261],
        [ -3.5578,   3.6268],
        [ -4.4798,   3.6140],
        [-18.1834,  -9.4486],
        [  5.3481,  16.7997],
        [ -9.4852,  -4.8822]], grad_fn=<AddmmBackward0>)


## Loss
Definir la funcion de loss

In [12]:
#Me ayuda a calcular el loss
import torch.nn.functional as F

In [13]:
#Loss con mean square , F tienen varias funciones pero yo solo uso la del mse
loss_function = F.mse_loss

In [14]:
predicciones = model(inputs)
loss = loss_function(predicciones, targets)
print(loss)

tensor(10065.8447, grad_fn=<MseLossBackward0>)


## OPtimizador
Manera que me ayuda a modificar automaticamente los pesos y sesgos

In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [16]:
#FUncion general que contiene todo lo anterior pero esta vez para entrenar

def training(epochs, model, loss_function, optimizer, train_dataloader):
  #Este es el ciclo de entrenamiento
  #iteramos sobre el numero de epochs de entrenamiento
  for epoch in range(epochs):

    for input, target in train_dataloader:

      #genero predicciones
      predictions = model(input)

      #calculo el loss
      loss = loss_function(predictions, target)

      #Calculo de gradiente
      loss.backward()

      #Ajuste de pesos y sesgos
      optimizer.step()

      #Reseteo de gradientes a 0
      optimizer.zero_grad()

    if((epoch+1)%10 == 0):
      print('epochs [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))


In [17]:
training(1000, model, loss_function, optimizer, train_dataloader)

epochs [10/1000], Loss: 1069.8171
epochs [20/1000], Loss: 1340.7288
epochs [30/1000], Loss: 288.1100
epochs [40/1000], Loss: 482.5612
epochs [50/1000], Loss: 133.5607
epochs [60/1000], Loss: 229.2972
epochs [70/1000], Loss: 73.7477
epochs [80/1000], Loss: 27.3150
epochs [90/1000], Loss: 23.0818
epochs [100/1000], Loss: 678.8109
epochs [110/1000], Loss: 651.3300
epochs [120/1000], Loss: 631.9965
epochs [130/1000], Loss: 36.6731
epochs [140/1000], Loss: 31.4364
epochs [150/1000], Loss: 601.0162
epochs [160/1000], Loss: 600.3275
epochs [170/1000], Loss: 13.0840
epochs [180/1000], Loss: 591.9799
epochs [190/1000], Loss: 27.6712
epochs [200/1000], Loss: 575.7699
epochs [210/1000], Loss: 17.1904
epochs [220/1000], Loss: 559.5034
epochs [230/1000], Loss: 566.4427
epochs [240/1000], Loss: 558.7565
epochs [250/1000], Loss: 40.9143
epochs [260/1000], Loss: 36.2884
epochs [270/1000], Loss: 551.0006
epochs [280/1000], Loss: 536.5843
epochs [290/1000], Loss: 527.9837
epochs [300/1000], Loss: 552.79

In [18]:
predicciones = model(inputs)
predicciones

tensor([[ 59.2026,  70.5242],
        [ 91.4952, 100.3366],
        [117.5317, 133.1944],
        [ 15.6614,  37.8672],
        [119.4805, 118.1731],
        [ 57.8703,  69.4381],
        [ 92.1973, 100.3960],
        [118.2858, 133.7715],
        [ 16.9937,  38.9534],
        [121.5148, 119.3187],
        [ 59.9046,  70.5836],
        [ 90.1629,  99.2505],
        [116.8297, 133.1350],
        [ 13.6271,  36.7217],
        [120.8128, 119.2593]], grad_fn=<AddmmBackward0>)

In [19]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [185., 119.],
        [ 57.,  69.],
        [ 82., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])